# Objective:
- Purpose of the project is to practice EDA using pandas

# Data source: 
- https://www.kaggle.com/competitions/nyc-taxi-trip-duration/data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read the data
data = pd.read_csv("train.csv")
print('Number of rows: ', data.shape[0])

Number of rows:  1458644


In [3]:
print('column names:', data.columns)

column names: Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'],
      dtype='object')
